<a href="https://colab.research.google.com/github/aljeshishe/notebooks/blob/master/deep_dream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from __future__ import print_function

from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
import scipy
import argparse

from keras import applications
from keras import backend as K


Using TensorFlow backend.


In [2]:
# Build the InceptionV3 network with our placeholder.
# The model will be loaded with pre-trained ImageNet weights.
model = applications.VGG16(include_top=False,
                           weights='imagenet')
for layer in model.layers:
  if 'conv' in layer.name:
    print(layer.name, layer.filters)

58892288/58889256 [==============================] - 1s 0us/step
block1_conv1 64
block1_conv2 64
block2_conv1 128
block2_conv2 128
block3_conv1 256
block3_conv2 256
block3_conv3 256
block4_conv1 512
block4_conv2 512
block4_conv3 512
block5_conv1 512
block5_conv2 512
block5_conv3 512


In [3]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import matplotlib.pyplot as plt

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    #x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def generate_pattern(layer_name, filter_index, size):
    layer = model.get_layer(layer_name)
    layer_output = layer.output
    
    loss = K.mean(layer_output[:, :, :, filter_index])
    grads = K.gradients(loss, model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([model.input], [loss, grads])
    step = 1
    rand_img = np.random.random((1, size, size, 3)) * 20 + 128.
    for _ in range(20):
        loss_value, grads_value = iterate([rand_img])
        rand_img += grads_value * step
    
    img = rand_img[0]
    return img


In [8]:
path = '/content/drive/My Drive/Colab Notebooks/deep_dream'
count = 20
size = 1024
for layer in model.layers:
  if 'conv' not in layer.name:
    continue
  count = min(count, layer.filters)
  layer_name = layer.name
  for i in range(count):
    print(f'{layer_name} {i}')
    img = generate_pattern(layer_name=layer_name, filter_index=i, size=size)
    #plt.figure(figsize=(10,10))
    #plt.imshow(deprocess_image(img))
    save_img(f'{path}/{layer_name}_{i}.jpg', deprocess_image(img))
    

block1_conv1 0
block1_conv1 1
block1_conv1 2
block1_conv1 3


KeyboardInterrupt: ignored

In [0]:
layer_name = 'block4_conv1'
i = 418
size = 1024 * 2
img = generate_pattern(layer_name, i, size)
save_img(f'{layer_name}_{i}_{size}.jpg', deprocess_image(img))